In [13]:
import pyModeS as pms
import pandas as pd
import base64

Lectura del fichero de despegues

In [58]:
datos  = pd.read_csv('test.csv', sep=';')
datos = datos.drop(columns=['Unnamed: 2'])
datos

,ts_kafka,message
0,1734667201534,oAANuYAVzS4ABLg3aB0=
1,1734667201534,kDQjSSgABjG4hW4fTbY=
2,1734667201534,XTxUM/+DTw==
3,1734667201534,kDQjShBRgxXDaCAWeDM=
4,1734667201534,jTRV1fgjAAIASbigrIs=
...,...,...
8354,1734667796655,IAAAALRFCw==
8355,1734667796655,IAAAALRFCw==
8356,1734667796655,jTlkQ5kItjeQCBj8Xms=
8357,1734667796655,IAAGM27mPA==


Los datos correspenden a la lectura que ha realizado la antena durante 1 minuto.

Son poco más de 8k filas, y la informacion de cada despeque se encuentra en la columna message, formateada en base 64.

Almacenaremos los datos en diccionarios.

In [60]:
for mensaje in datos['message']:
  msg_hex = base64.b64decode(mensaje).hex().upper()
  datos.loc[datos['message'] == mensaje, 'hexadecimal'] = msg_hex

In [41]:
datos.head()

,ts_kafka,message,hexaecimal,hexadecimal
0,1734667201534,oAANuYAVzS4ABLg3aB0=,A0000DB98015CD2E0004B837681D,A0000DB98015CD2E0004B837681D
1,1734667201534,kDQjSSgABjG4hW4fTbY=,9034234928000631B8856E1F4DB6,9034234928000631B8856E1F4DB6
2,1734667201534,XTxUM/+DTw==,5D3C5433FF834F,5D3C5433FF834F
3,1734667201534,kDQjShBRgxXDaCAWeDM=,9034234A10518315C36820167833,9034234A10518315C36820167833
4,1734667201534,jTRV1fgjAAIASbigrIs=,8D3455D5F82300020049B8A0AC8B,8D3455D5F82300020049B8A0AC8B


ModeS supports the decoding of following types of messages:

DF4 / DF20: Altitude code

DF5 / DF21: Identity code (squawk code)

DF17 / DF18: Automatic Dependent Surveillance-Broadcast (ADS-B)

TC=1-4 / BDS 0,8: Aircraft identification and category

TC=5-8 / BDS 0,6: Surface position

TC=9-18 / BDS 0,5: Airborne position

TC=19 / BDS 0,9: Airborne velocity

TC=28 / BDS 6,1: Airborne status [to be implemented]

TC=29 / BDS 6,2: Target state and status information [to be implemented]

TC=31 / BDS 6,5: Aircraft operational status [to be implemented]

DF20 / DF21: Mode-S Comm-B messages

BDS 1,0: Data link capability report

BDS 1,7: Common usage GICB capability report

BDS 2,0: Aircraft identification

BDS 3,0: ACAS active resolution advisory

BDS 4,0: Selected vertical intention

BDS 4,4: Meteorological routine air report (experimental)

BDS 4,5: Meteorological hazard report (experimental)

BDS 5,0: Track and turn report

BDS 6,0: Heading and speed report

In [62]:
# Los datos tienen registros de aviones, y la informacion esta separada en distintos typecodes
# Lo primero es clasificar por DF

datos_id = datos.copy()
datos_id["DF"] = datos["hexadecimal"].map(pms.df)
datos_id["ICAO"] = datos["hexadecimal"].map(pms.icao)
datos_id["TYPECODE"] = datos_id.apply(lambda x: pms.adsb.typecode(x["hexadecimal"]) if x["DF"] in [17, 18] else None, axis=1)
datos_id["BDS"] = datos_id.apply(lambda x: pms.bds.infer(x["hexadecimal"]) if x["DF"] in [20, 21] else None, axis=1)

datos_id

,ts_kafka,message,hexadecimal,DF,ICAO,TYPECODE,BDS
0,1734667201534,oAANuYAVzS4ABLg3aB0=,A0000DB98015CD2E0004B837681D,20,AA5F41,NaN,BDS50
1,1734667201534,kDQjSSgABjG4hW4fTbY=,9034234928000631B8856E1F4DB6,18,342349,5.0,None
2,1734667201534,XTxUM/+DTw==,5D3C5433FF834F,11,3C5433,NaN,None
3,1734667201534,kDQjShBRgxXDaCAWeDM=,9034234A10518315C36820167833,18,34234A,2.0,None
4,1734667201534,jTRV1fgjAAIASbigrIs=,8D3455D5F82300020049B8A0AC8B,17,3455D5,31.0,None
...,...,...,...,...,...,...,...
8354,1734667796655,IAAAALRFCw==,20000000B4450B,4,342354,NaN,None
8355,1734667796655,IAAAALRFCw==,20000000B4450B,4,342354,NaN,None
8356,1734667796655,jTlkQ5kItjeQCBj8Xms=,8D3964439908B637900818FC5E6B,17,396443,19.0,None
8357,1734667796655,IAAGM27mPA==,200006336EE63C,4,346419,NaN,None


In [34]:

msg = '9034234928000631B8856E1F4DB6'    # Convert grey code to integer


pms.typecode(msg)
pms.df(msg)

pms.adsb.surface_velocity(msg)



(None, None, 0, 'GS')

In [57]:
pms.tell("A800139D8031DB36FFFCDA676570")

                     Message: A800139D8031DB36FFFCDA676570 
                ICAO address: 4B18F5 
             Downlink Format: 21 
                    Protocol: Mode-S Comm-B identity reply 
                 Squawk code: 6257 
                         BDS: BDS50 (Track and turn report) 
                  Roll angle: 0.17578125 degrees
                 Track angle: 41.66015625 degrees
                  Track rate: -0.03125 degree/second
                Ground speed: 438 knots
               True airspeed: 436 knots
